In [60]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
from math import radians, sin, cos, sqrt, atan2

In [61]:
care_homes_df = pd.read_csv('/Users/nateoppenheimer/code/willbanny/Location-Analysis/carehome_locations.csv')
london_grid_coords = pd.read_csv('/Users/nateoppenheimer/code/willbanny/Location-Analysis/London_Data_Expanded_1.csv')

In [62]:
care_homes_df = care_homes_df.rename({'latitude': 'lat', 'longitude': 'lng'}, axis=1)
care_homes_df.head()

,Unnamed: 0,Location_Names,lat,lng,num_beds,town_city,region,carehome_y_n
0,0,Henley House,52.088019,1.150452,66.0,Ipswich,East,Y
1,5,Charlotte House,53.345515,-3.002580,103.0,Wirral,North West,Y
2,6,Regency Care Centre,53.558464,-2.295343,60.0,Bury,North West,Y
3,10,Kingswood House Nursing Home,50.857483,0.561912,22.0,St Leonards On Sea,South East,Y
4,13,Little Haven,51.442094,0.002853,15.0,London,London,Y


In [63]:
care_homes_df['lat'] = pd.to_numeric(care_homes_df['lat'], errors='coerce')
care_homes_df['lng'] = pd.to_numeric(care_homes_df['lng'], errors='coerce')
care_homes_df.info()
# df['colname'] = pd.to_numeric(df['colname'], errors='coerce')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20808 entries, 0 to 20807
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      20808 non-null  int64  
 1   Location_Names  20808 non-null  object 
 2   lat             20803 non-null  float64
 3   lng             20803 non-null  float64
 4   num_beds        20780 non-null  float64
 5   town_city       20808 non-null  object 
 6   region          20788 non-null  object 
 7   carehome_y_n    20808 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 1.3+ MB


In [64]:
care_homes_df = care_homes_df.dropna()
care_homes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20760 entries, 0 to 20807
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      20760 non-null  int64  
 1   Location_Names  20760 non-null  object 
 2   lat             20760 non-null  float64
 3   lng             20760 non-null  float64
 4   num_beds        20760 non-null  float64
 5   town_city       20760 non-null  object 
 6   region          20760 non-null  object 
 7   carehome_y_n    20760 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 1.4+ MB


In [65]:
classified_coords = london_grid_coords[['lat', 'lng', 'district_name']].copy()
classified_coords.head()

,lat,lng,district_name
0,51.607913,-0.355127,Harrow London Boro
1,51.607913,-0.351505,Harrow London Boro
2,51.605664,-0.351507,Harrow London Boro
3,51.603414,-0.402215,Harrow London Boro
4,51.601165,-0.402215,Harrow London Boro


In [71]:
care_homes_df = care_homes_df[care_homes_df['region'] == 'London'].reset_index(drop=True).drop(columns=['Unnamed: 0'])

In [72]:
bt = BallTree(np.deg2rad(classified_coords[['lat', 'lng']].values), metric='haversine')

In [73]:
distances, indices = bt.query(np.deg2rad(care_homes_df[['lat', 'lng']]))

dep_engineered = classified_coords.iloc[indices[:, 0]][['district_name']]
dep_engineered[['lng', 'lat']] = care_homes_df[['lng', 'lat']].values

In [74]:
dep_engineered.reset_index(drop=True, inplace=True)

In [75]:
dep_engineered.to_csv('care_homes_by_district.csv')

In [76]:
dep_engineered.head()

,district_name,lng,lat
0,Lewisham London Boro,0.002853,51.442094
1,Kingston upon Thames London Boro,-0.253510,51.392733
2,Greenwich London Boro,0.081430,51.484673
3,Wandsworth London Boro,-0.182608,51.458354
4,Newham London Boro,0.066140,51.512515


In [77]:
dep_engineered.columns = list(dep_engineered.columns)

In [78]:
dep_engineered[dep_engineered['district_name']=='Redbridge London Boro'].reset_index(drop=True).copy()

,district_name,lng,lat
0,Redbridge London Boro,0.107145,51.569384
1,Redbridge London Boro,0.022524,51.573877
2,Redbridge London Boro,0.114437,51.563273
3,Redbridge London Boro,0.093867,51.560700
4,Redbridge London Boro,0.063702,51.565696
...,...,...,...
89,Redbridge London Boro,0.096333,51.562078
90,Redbridge London Boro,0.100804,51.561117
91,Redbridge London Boro,0.017849,51.568679
92,Redbridge London Boro,0.089859,51.559037
